# Decision Tree and Random Forest

In [1]:
# Install imblearn library (https://imbalanced-learn.org/stable/) for handling imbalanced data set.
#!pip install imblearn

# Install graphviz library (https://pypi.org/project/graphviz/) for visualizing the decision tree.
#!pip install graphviz

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Load the data
df = pd.read_excel("HR_Employee_Data.xlsx")

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
# Show several data samples
df.head()

This dataset is for employee churn prediction based on the employee features so that HR department can have a suitable reaction.

* Emp_Id: id of Employee.
* satisfaction_level: satisfaction level of employee in percentage. 100% or 1 is very satisfied. 0% or 0 is not satisfied.
* last_evaluation: time from the last performance evaluation.
* number_project: number of projects an employee is working on.
* average_montly_hours: average working hours of an employee in the last 3 months.
* time_spend_company: time spent by an employee on commuting to office.
* Work_accident: if an employee was involved in a work accident.
* promotion_last_5years: if an employee has a promotion in past 5 years.
* Department: Department employee is working in.
* left: if the employee has left the company (target value)

In [ ]:
print("Data shape: " + str(df.shape) + "\n")     # Shape of data frame
df.head()     # Show some data samples

In [ ]:
data = df.copy()     # Create a copy of the data frame

In [ ]:
# Drop Employer ID in two ways
# data = data.drop(columns=["Emp_Id"])              # inplace = False: drop and assign to the variable 'data'
data.drop(columns=["Emp_Id"], inplace=True)         # inplace = True: directly drop the feature in the 'data' variable.

In [ ]:
data.head()     # Show some data samples

In [ ]:
# Transform 'Department' and 'Salary' variable to one-hot encoding variables
data = pd.get_dummies(data)
data

In [ ]:
# Drop the column 'left' because it is the data label
X = data.drop(columns=["left"])

# Assign data label to to variable 'y'
y = data.left

In [ ]:
# Load libraries
from sklearn.model_selection import train_test_split, GridSearchCV

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=1, test_size=0.2)

In [ ]:
# Before oversampling
print("Before oversampling: " + str(X_train.shape))
print(np.unique(y_train, return_counts=True))                  # Print number of labels, label '0' dominates '1' 

In [ ]:
# Apply oversampling method for label '1' (SMOTE will generate samples having label '1' such that the number of samples is equal to label 'y' in the training set).
# Imbalanced-learn (imported as imblearn) provides tools when dealing with classification with imbalanced classes.
# Reference: https://imbalanced-learn.org/
from imblearn.over_sampling import SMOTE     # Load the SMOTE library
smote = SMOTE(random_state=5)                # Initialize SMOTE
X_train_oversampling, y_train_oversampling = smote.fit_resample(X_train, y_train)     # Oversample label '1' (minority class) in the training set

Synthetic Minority Oversampling Technique (SMOTE) is a machine learning technique that solves problems occuring when using an imbalanced data set. Here follows how the SMOTE algorithm works:

* Step 1: A random example from the minority class is first chosen.
* Step 2: Then K of the nearest neighbors for that example are found (typically K=5).
* Step 3: A randomly selected neighbor is chosen a mong the k neighbors.
* Step 4: A synthetic minority example is created at a random point between the random example (Step 1) and the neighbor (Step 3) in feature space.

Reference: N. V. Chawla, K. W. Bowyer, L. O. Hall, W. P. Kegelmeyer, "SMOTE: Synthetic Minority Over-sampling Technique", https://arxiv.org/abs/1106.1813


In [ ]:
# Before oversampling
print("Before oversampling: " + str(X_train.shape))
print(np.unique(y_train, return_counts=True))                  # Print number of labels, label '0' dominates '1' 

# After oversampling
print("After oversampling: " + str(X_train_oversampling.shape))
print(np.unique(y_train_oversampling, return_counts=True))     # Print number of labels, now label '0' and '1' have the same number of labels.

In [ ]:
# Import GridSearchCV for finding the best hyper-parameter set.
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

params = {"criterion": ["gini", "entropy"],             # Criterion to evaluate the purity.
         "max_depth": [3, 5],                           # Maximum depth of the tree
         "min_samples_split": [4, 8]}                   # Stop splitting condition.

grid_search = GridSearchCV(estimator=DecisionTreeClassifier(), param_grid=params, cv=5)

Gini impurity is one of the methods used in decision tree algorithms to decide the optimal split. Gini index calculates the amount of probability of a specific feature that is classified incorrectly when selected randomly. If all the elements are linked with a single class then it is called pure. It ranges from 0-1.
* Gini = 0: all elements
* Gini = 1: Randomly distributed
* Gini = 0.5: equally distributed

In [ ]:
# Run the search on oversampled training data samples.
grid_search.fit(X_train_oversampling, y_train_oversampling)

In [ ]:
# Best set of hyper-parameters found after searching
grid_search.best_params_

In [ ]:
# Build a decision tree model pipeline from the best set of hyper-parameters found
model_dt = DecisionTreeClassifier(criterion=grid_search.best_params_['criterion'], max_depth=grid_search.best_params_['max_depth'], min_samples_split=grid_search.best_params_['min_samples_split'])

In [ ]:
# Train the decision tree model
model_dt.fit(X_train_oversampling, y_train_oversampling)

In [ ]:
# Calculate some evaluation metrics: precision, recall, f1-score. 'Support' is the number of actual occurrences of the class in the dataset.
from sklearn.metrics import classification_report

# Make prediction on the original test set (after training on the over-sampled training set).
pred_y = model_dt.predict(X_test)

print(classification_report(y_test, pred_y))

# Note 1: macro avg = (precision_0 + precision_1) / 2
# Note 2: weighted avg = (support_0 * precision_0 + support_1 * precision_1) / number_of_samples

In [ ]:
# Building a RandomForest is similar to building a decision tree, but there is an additional hyper-parameter called n_estimators which is the number of trees in the forest.
from sklearn.ensemble import RandomForestClassifier

params = {"criterion": ["gini", "entropy"],             # Criterion to evaluate the purity.
         "max_depth": [7, 9, 11],                           # Maximum depth of the tree
         "min_samples_split": [8, 12, 16]}                   # Stop splitting condition.

grid_search_rf = GridSearchCV(estimator=RandomForestClassifier(n_estimators=10, n_jobs=10), param_grid=params, cv= 5) # Number of trees in the forest is 10

# Run the search on oversampled training data samples.
grid_search_rf.fit(X_train_oversampling, y_train_oversampling)     # Train the RandomForest

In [ ]:
# Best set of hyper parameters of the Random Forest
grid_search_rf.best_params_

In [ ]:
# Build a Random Forest model pipeline from the best set of hyper-parameters found
model_rf = RandomForestClassifier(n_estimators=10, random_state=1, criterion=grid_search_rf.best_params_['criterion'], max_depth=grid_search_rf.best_params_['max_depth'], min_samples_split=grid_search_rf.best_params_['min_samples_split'])     # Initialize the RandomForest

In [ ]:
# Train the Random Forest model
model_rf.fit(X_train_oversampling, y_train_oversampling)

In [ ]:
# Calculate some evaluation metrics: precision, recall, f1-score. 'Support' is the number of actual occurrences of the class in the dataset.
from sklearn.metrics import classification_report

# Make prediction on the original test set (after training on the over-sampled training set).
pred_y = model_rf.predict(X_test)

print(classification_report(y_test, pred_y))

# Note: theoretically saying, there is no guarantee that the Random Forest is alsways better than the Decision Tree, i.e., the model ensembling is not always better than a single model in general.

## Feature Importance

Feature importance is calculated as the decrease in node impurity (by Mean Square Error - MSE) weighted by the probability of reaching that node. The node probability can be calculated by the number of samples that reach the node, divided by the total number of samples. The feature importance is the sum of importance scores of all nodes using that feature to split data samples. The higher the sum value the more important the feature.

In [ ]:
# Print the importance level of each feature
weights = model_dt.feature_importances_     # Feature importance scores.
cols = X.columns                            # Corresponding names of important features.
feature_importances = pd.Series(weights, index=cols)     # Generate Series containing importance level
print(feature_importances)

# Plot the five most important features 
feature_importances.nlargest(5).plot(kind="barh")

In [ ]:
# Visualize the tree. Note: to run dot command, we have to install 'graphviz'
from sklearn import tree
tree.export_graphviz(decision_tree=model_dt, out_file='classification-tree.dot', feature_names=X.columns)     # Save model as 'classification-tree.dot'
from subprocess import call
call(['dot', '-Tpng', 'classification-tree.dot', '-o', 'classification-tree.png', '-Gdpi=300'])        # Convert the model to an image

In [ ]:
from IPython.display import Image
Image(filename = 'classification-tree.png')     # Load and display file 'classification-tree.png'

## Classification decision tree vs. Regression decision tree

There are 2 types of Decision trees: 
* Classification trees are used when output labels are categorical. A classification tree splits the dataset based on the homogeneity of data. **Measures of impurity like entropy or Gini are used to quantify the homogeneity** of the data when it comes to classification trees. Reference: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier
* Regression trees are used when output labels are numerical. Since the tree wants to predict continuous variables, it cannot calculate the entropy and go through the same process. The data is split at several points for each independent variable. At those points, the **error between the predicted values and actual values is squared to get the Sum of Squared Errors (SSE)**. The point that has the lowest SSE is chosen as the split point. This process is continued recursively. In the Regression Tree algorithm, we do the same thing as in the Classification trees, but we try to reduce the squared error at each child rather than the entropy. Reference: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html

To sum up, the basic difference between the two types of tree is how to choose the best feature to split. Classification tree chooses the feature that minimizes entropy/gini of the split datasets, whereas Regression tree wants to minimize the squared error between the predicted values and actual values (label).

In [ ]:
# Initialize regression tree model
model_dt_reg = tree.DecisionTreeRegressor(criterion='squared_error', max_depth=grid_search.best_params_['max_depth'], min_samples_split=grid_search.best_params_['min_samples_split'])
model_dt_reg.fit(X_train_oversampling, y_train_oversampling)     

In [ ]:
# Visualize the regression tree model
dot_data = tree.export_graphviz(decision_tree=model_dt_reg, out_file='regression-tree.dot', feature_names=X.columns)     # Save model as 'regression-tree.dot'
call(['dot', '-Tpng', 'regression-tree.dot', '-o', 'regression-tree.png', '-Gdpi=300'])         # Convert the model to an image

Image(filename = 'regression-tree.png')     # Load and display file 'regression-tree.png'

## Feature importance

Feature importance is calculated as the decrease in node impurity weighted by the probability of reaching that node. The node probability can be calculated by the number of samples that reach the node, divided by the total number of samples. The higher the value the more important the feature.

Node impurity can be Entropy, Gini (in classification trees) or Squared error (in regression tree). Here follows two examples of the feature importance calculation based on Entropy and Gini.

### Example of feature importance calculation based on ENTROPY impurity

In [ ]:
# Build a classification tree based on Entropy
from sklearn.tree import DecisionTreeClassifier
model_fi_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=2, min_samples_split=10)
model_fi_entropy.fit(X_train_oversampling, y_train_oversampling)

In [ ]:
# Visualize the tree. Note: to run dot command, we have to install 'graphviz'
from sklearn import tree
tree.export_graphviz(decision_tree=model_fi_entropy, out_file='classification-tree-entropy.dot', feature_names=X.columns)     # Save model as 'classification-tree-entropy.dot'
from subprocess import call
call(['dot', '-Tpng', 'classification-tree-entropy.dot', '-o', 'classification-tree-entropy.png', '-Gdpi=300'])        # Convert the model to an image

In [ ]:
from IPython.display import Image
Image(filename = 'classification-tree-entropy.png')     # Load and display file 'classification-tree.png'

In [ ]:
# Print the importance level of each feature
weights_entropy = model_fi_entropy.feature_importances_     # Feature importance scores.
cols = X.columns                                            # Corresponding names of important features.
feature_importances_entropy = pd.Series(weights_entropy, index=cols)     # Generate Series containing importance level
print(feature_importances_entropy)

# Plot the five most important features 
feature_importances_entropy.nlargest(5).plot(kind="barh")

## Feature importance

### Example of feature importance calculation based on GINI impurity

In [ ]:
# Build a classification tree based on Entropy
from sklearn.tree import DecisionTreeClassifier
model_fi_gini = DecisionTreeClassifier(criterion='gini', max_depth=2, min_samples_split=10)
model_fi_gini.fit(X_train_oversampling, y_train_oversampling)

In [ ]:
# Visualize the tree. Note: to run dot command, we have to install 'graphviz'
from sklearn import tree
tree.export_graphviz(decision_tree=model_fi_gini, out_file='classification-tree-gini.dot', feature_names=X.columns)     # Save model as 'classification-tree-gini.dot'
from subprocess import call
call(['dot', '-Tpng', 'classification-tree-gini.dot', '-o', 'classification-tree-gini.png', '-Gdpi=300'])        # Convert the model to an image

In [ ]:
from IPython.display import Image
Image(filename = 'classification-tree-gini.png')     # Load and display file 'classification-tree-gini.png'

In [ ]:
# Print the importance level of each feature
weights_gini = model_fi_gini.feature_importances_           # Feature importance scores.
cols = X.columns                                            # Corresponding names of important features.
feature_importances_gini = pd.Series(weights_gini, index=cols)   # Generate Series containing importance level
print(feature_importances_gini)

# Plot the five most important features 
feature_importances_gini.nlargest(5).plot(kind="barh")